# shorturl.at/iOY35

Основано на: https://github.com/DanAnastasyev/DeepNLP-Course Week 12

In [1]:
!git clone https://github.com/MiuLab/SlotGated-SLU.git
!wget -qq https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week08_multitask/conlleval.py

Cloning into 'SlotGated-SLU'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 51 (delta 4), reused 2 (delta 2), pack-reused 44
Unpacking objects: 100% (51/51), 426.19 KiB | 3.77 MiB/s, done.


In [2]:
# !pip install torchtext==0.10.0
!pip install  torchtext==0.4.0 torch==1.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 63.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.13.1 which 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
DEVICE = torch.device('cuda')

# Диалоговые системы

Диалоговые системы делятся на два типа - *goal-orientied* и *general conversation*.

**General conversation** - это болталка, разговор на свободную тему:  
<img src="https://i.ibb.co/bFwwGpc/alice.jpg" width="200"/>

Сегодня будем говорить не про них, а про **goal-orientied** системы:

<img src="https://hsto.org/webt/gj/3y/xl/gj3yxlqbr7ujuqr9r2akacxmkee.jpeg" width="600"/>

*From [Как устроена Алиса](https://habr.com/company/yandex/blog/349372/)*

Пользователь говорит что-то, это что-то распознается. По распознанному определяется - что, где и когда он хотел. Дальше диалоговый движок решает, действительно ли пользователь знает, чего хотел попросить. Происходит поход в источники - узнать информацию, которую (кажется) запросил пользователь. Исходя из всего этого генерируется некоторый ответ:

<img src="https://i.ibb.co/8XcdpJ7/goal-orientied.png" width="600"/>

*From [Как устроена Алиса](https://habr.com/company/yandex/blog/349372/)*

Будем учить ту часть, которая посередине - классификатор и теггер. Всё остальное обычно - эвристики и захардкоженные ответы.

## Данные

Есть условно стандартный датасет - atis, который неприлично маленький, на самом деле.

К нему можно взять еще датасет snips - он больше и разнообразнее.

Оба датасета возьмем из репозитория статьи [Slot-Gated Modeling for Joint Slot Filling and Intent Prediction](http://aclweb.org/anthology/N18-2118).

Начнем с atis.

In [2]:
import os 

def read_dataset(path):
    with open(os.path.join(path, 'seq.in')) as f_words, \
         open(os.path.join(path, 'seq.out')) as f_tags, \
         open(os.path.join(path, 'label')) as f_intents:
        
        return [  (words.strip().split(), tags.strip().split(), intent.strip()) 
                  for words, tags, intent in zip(f_words, f_tags, f_intents)
               ]

In [3]:
train_data = read_dataset('SlotGated-SLU/data/atis/train/')
val_data = read_dataset('SlotGated-SLU/data/atis/valid/')
test_data = read_dataset('SlotGated-SLU/data/atis/test/')

In [4]:
intent_to_example = {example[2]: example for example in train_data}
for example in intent_to_example.values():
    print('Intent:\t', example[2])
    print('Text:\t', '\t'.join(example[0]))
    print('Tags:\t', '\t'.join(example[1]))
    print()

Intent:	 atis_flight
Text:	 is	there	a	delta	flight	from	denver	to	san	francisco
Tags:	 O	O	O	B-airline_name	O	O	B-fromloc.city_name	O	B-toloc.city_name	I-toloc.city_name

Intent:	 atis_airfare
Text:	 what	is	the	most	expensive	one	way	fare	from	boston	to	atlanta	on	american	airlines
Tags:	 O	O	O	B-cost_relative	I-cost_relative	B-round_trip	I-round_trip	O	O	B-fromloc.city_name	O	B-toloc.city_name	O	B-airline_name	I-airline_name

Intent:	 atis_airline
Text:	 list	airlines	serving	between	denver	and	san	francisco
Tags:	 O	O	O	O	B-fromloc.city_name	O	B-toloc.city_name	I-toloc.city_name

Intent:	 atis_ground_service
Text:	 tell	me	about	ground	transportation	between	orlando	international	and	orlando
Tags:	 O	O	O	O	O	O	B-fromloc.airport_name	I-fromloc.airport_name	O	B-toloc.city_name

Intent:	 atis_quantity
Text:	 how	many	airlines	have	flights	with	service	class	yn
Tags:	 O	O	O	O	O	O	O	O	B-fare_basis_code

Intent:	 atis_city
Text:	 where	is	lester	pearson	airport
Tags:	 O	O	B-airport_name	

In [5]:
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator

tokens_field = Field()
tags_field = Field(unk_token=None)
intent_field = LabelField()

fields = [('tokens', tokens_field), ('tags', tags_field), ('intent', intent_field)]

train_dataset = Dataset([Example.fromlist(example, fields) for example in train_data], fields)
val_dataset = Dataset([Example.fromlist(example, fields) for example in val_data], fields)
test_dataset = Dataset([Example.fromlist(example, fields) for example in test_data], fields)
# ВНИМАНИЕ - КОСТЫЛЬ
all_dataset = Dataset([Example.fromlist(example, fields) for example in train_data+val_data+test_data], fields)

tokens_field.build_vocab(train_dataset)
tags_field.build_vocab(all_dataset)
intent_field.build_vocab(all_dataset)

print('Vocab size =', len(tokens_field.vocab))
print('Tags count =', len(tags_field.vocab))
print('Intents count =', len(intent_field.vocab))

train_iter, val_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, val_dataset, test_dataset), batch_sizes=(32, 128, 128), 
    shuffle=True, device=DEVICE, sort=False
)

Vocab size = 869
Tags count = 128
Intents count = 26


То же самое со snips

In [6]:
snips_train_data = read_dataset('SlotGated-SLU/data/snips/train/')
snips_val_data = read_dataset('SlotGated-SLU/data/snips/valid/')
snips_test_data = read_dataset('SlotGated-SLU/data/snips/test/')
snips_intent_to_example = {example[2]: example for example in snips_train_data}
for example in snips_intent_to_example.values():
    print('Intent:\t', example[2])
    print('Text:\t', '\t'.join(example[0]))
    print('Tags:\t', '\t'.join(example[1]))
    print()

Intent:	 PlayMusic
Text:	 play	funky	heavy	bluesy
Tags:	 O	B-playlist	I-playlist	I-playlist

Intent:	 AddToPlaylist
Text:	 add	gabrial	mcnair	to	my	love	in	paris	list
Tags:	 O	B-artist	I-artist	O	B-playlist_owner	B-playlist	I-playlist	I-playlist	O

Intent:	 RateBook
Text:	 rate	richard	carvel	4	out	of	6
Tags:	 O	B-object_name	I-object_name	B-rating_value	O	O	B-best_rating

Intent:	 SearchScreeningEvent
Text:	 can	i	get	the	movie	schedule	for	loews	cineplex	entertainment
Tags:	 O	O	O	O	B-object_type	I-object_type	O	B-location_name	I-location_name	I-location_name

Intent:	 BookRestaurant
Text:	 i	want	to	eat	choucroute	at	a	brasserie	for	8
Tags:	 O	O	O	O	B-served_dish	O	O	B-restaurant_type	O	B-party_size_number

Intent:	 GetWeather
Text:	 tell	me	when	it	ll	be	chillier	in	cavalero	corner	id
Tags:	 O	O	O	O	O	O	B-condition_temperature	O	B-city	I-city	B-state

Intent:	 SearchCreativeWork
Text:	 go	to	the	photograph	the	inflated	tear
Tags:	 O	O	O	B-object_type	B-object_name	I-object_name	I-o

In [7]:
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator

snips_tokens_field = Field()
snips_tags_field = Field(unk_token=None)
snips_intent_field = LabelField()

fields = [('tokens', snips_tokens_field), ('tags', snips_tags_field), ('intent', snips_intent_field)]

snips_train_dataset = Dataset([Example.fromlist(example, fields) for example in snips_train_data], fields)
snips_val_dataset = Dataset([Example.fromlist(example, fields) for example in snips_val_data], fields)
snips_test_dataset = Dataset([Example.fromlist(example, fields) for example in snips_test_data], fields)
# ВНИМАНИЕ - КОСТЫЛЬ
snips_all_dataset = Dataset([Example.fromlist(example, fields) for example in snips_train_data+snips_val_data+snips_test_data], fields)

snips_tokens_field.build_vocab(snips_train_dataset)
snips_tags_field.build_vocab(snips_all_dataset)
snips_intent_field.build_vocab(snips_all_dataset)

print('Vocab size =', len(snips_tokens_field.vocab))
print('Tags count =', len(snips_tags_field.vocab))
print('Intents count =', len(snips_intent_field.vocab))

snips_train_iter, snips_val_iter, snips_test_iter = BucketIterator.splits(
    datasets=(snips_train_dataset, snips_val_dataset, snips_test_dataset), batch_sizes=(32, 128, 128), 
    shuffle=True, device=DEVICE, sort=False
)

Vocab size = 11420
Tags count = 73
Intents count = 7


## Классификатор интентов

Начнем с классификатора: к какому интенту относится данный запрос.

Ничего умного - берём rnn'ку и учимся предсказывать метки-интенты.

In [8]:
class IntentClassifierModel(nn.Module):
    def __init__(self, vocab_size, intents_count, emb_dim=64,
                 lstm_hidden_dim=128, num_layers=1, dropout_p=0.2):
        super().__init__()

        self.embeddings_layer = nn.Embedding(vocab_size, emb_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.lstm_layer = nn.LSTM(emb_dim, lstm_hidden_dim, batch_first=True,
                                  bidirectional=True, num_layers=num_layers)
        self.out_layer = nn.Linear(lstm_hidden_dim * 2, intents_count)

    def forward(self, inputs):
        projections = self.embeddings_layer(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        output, (final_hidden_state, _) = self.lstm_layer(projections)
        hidden = self.dropout(torch.cat((final_hidden_state[0], final_hidden_state[1]), dim=1))
        output = self.out_layer.forward(hidden)
        return output

In [9]:
class ModelTrainer():
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        self.epoch_loss = 0
        self.correct_count, self.total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        self.model.train(is_train)
        
    def on_epoch_end(self):
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.correct_count / self.total_count
        )
        
    def on_batch(self, batch):
        logits = self.model(batch.tokens.transpose(0, 1))
        loss = self.criterion(logits, batch.intent)
        predicted_intent = torch.max(logits, axis=1)[1]
        self.total_count += predicted_intent.size(0)
        self.correct_count += torch.sum(predicted_intent == batch.intent).item()
        if self.is_train:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
        self.epoch_loss += loss.item()

In [10]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []

def do_epoch(trainer, data_iter, is_train, name=None):
    trainer.on_epoch_begin(is_train, name, batches_count=len(data_iter))
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=trainer.batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):
                batch_progress = trainer.on_batch(batch)

                progress_bar.update()
                progress_bar.set_description(batch_progress)
                
            epoch_progress = trainer.on_epoch_end()
            progress_bar.set_description(epoch_progress)
            progress_bar.refresh()
          
def fit(trainer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        do_epoch(trainer, train_iter, is_train=True, name=name_prefix + 'Train:')
        
        if not val_iter is None:
            do_epoch(trainer, val_iter, is_train=False, name=name_prefix + '  Val:')

In [ ]:
model = IntentClassifierModel(vocab_size=len(tokens_field.vocab), intents_count=len(intent_field.vocab)).to(DEVICE)
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())
trainer = ModelTrainer(model, criterion, optimizer)
fit(trainer, train_iter, epochs_count=30, val_iter=val_iter)

In [ ]:
do_epoch(trainer, test_iter, is_train=False, name='Test:')

 14%|█▍        | 1/7 [00:00<00:00, 102.83it/s]

 71%|███████▏  | 5/7 [00:00<00:00, 107.50it/s]

Test: Loss = 0.34127, Accuracy = 95.07%: 100%|██████████| 7/7 [00:00<00:00, 103.33it/s]

Test: Loss = 0.34127, Accuracy = 95.07%: 100%|██████████| 7/7 [00:00<00:00, 94.32it/s]


## Теггер

![](https://commons.bmstu.wiki/images/0/00/NER1.png)  
*From [NER](https://ru.bmstu.wiki/NER_(Named-Entity_Recognition)*

#### **Задание 1.1**
Напишите простой теггер

In [ ]:
class TokenTaggerModel(nn.Module):
    def __init__(self, vocab_size, tags_count, emb_dim=64,
                 lstm_hidden_dim=128, num_layers=1, dropout_p=0.2):
        super().__init__()
        # YOUR CODE HERE
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.dropout = nn.Dropout(0.2)
        self.lstm = nn.LSTM(emb_dim, lstm_hidden_dim, batch_first=True,
                            num_layers=num_layers, bidirectional=True)
        self.linear = nn.Linear(2*lstm_hidden_dim, tags_count)

    def forward(self, inputs):
        # YOUR CODE HERE
        emb = self.embedding(inputs)
        drp = self.dropout(emb)
        output, (hidden_state, _) = self.lstm(drp)
        output = self.linear(output)
        return output

#### **Задание 1.2**
Обновите `ModelTrainer`: считать нужно всё те же лосс и accuracy, только теперь немного по-другому.

In [ ]:
class TagModelTrainer():
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        self.epoch_loss = 0
        self.correct_count, self.total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        self.model.train(is_train)
        
    def on_epoch_end(self):
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.correct_count / self.total_count
        )
        
    def on_batch(self, batch):
        # YOUR CODE HERE
        logits = self.model(batch.tokens.transpose(0, 1))
        true_tags = batch.tags.transpose(0, 1)
        loss = self.criterion(logits.transpose(1, 2), true_tags)
        predicted_tags = torch.max(logits, axis=2).indices
        self.total_count += torch.sum(true_tags!=0).item()
        self.correct_count += torch.sum(predicted_tags==true_tags).item() - torch.sum(true_tags==0).item()

        if self.is_train:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
        self.epoch_loss += loss.item()

In [ ]:
model = TokenTaggerModel(vocab_size=len(tokens_field.vocab), tags_count=len(tags_field.vocab)).to(DEVICE)
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())
trainer = TagModelTrainer(model, criterion, optimizer)
fit(trainer, train_iter, epochs_count=30, val_iter=val_iter)

[1 / 30] Train: Loss = 1.04754, Accuracy = 62.38%: 100%|██████████| 140/140 [00:01<00:00, 105.89it/s]
[1 / 30]   Val: Loss = 0.33551, Accuracy = 81.24%: 100%|██████████| 4/4 [00:00<00:00, 74.61it/s]
[2 / 30] Train: Loss = 0.29538, Accuracy = 87.20%: 100%|██████████| 140/140 [00:01<00:00, 110.59it/s]
[2 / 30]   Val: Loss = 0.15431, Accuracy = 90.79%: 100%|██████████| 4/4 [00:00<00:00, 89.36it/s]
[3 / 30] Train: Loss = 0.15983, Accuracy = 93.01%: 100%|██████████| 140/140 [00:01<00:00, 136.55it/s]
[3 / 30]   Val: Loss = 0.10306, Accuracy = 94.04%: 100%|██████████| 4/4 [00:00<00:00, 80.78it/s]
[4 / 30] Train: Loss = 0.10893, Accuracy = 95.20%: 100%|██████████| 140/140 [00:01<00:00, 129.69it/s]
[4 / 30]   Val: Loss = 0.06965, Accuracy = 95.49%: 100%|██████████| 4/4 [00:00<00:00, 86.47it/s]
[5 / 30] Train: Loss = 0.07767, Accuracy = 96.53%: 100%|██████████| 140/140 [00:01<00:00, 136.17it/s]
[5 / 30]   Val: Loss = 0.06755, Accuracy = 96.12%: 100%|██████████| 4/4 [00:00<00:00, 88.43it/s] 
[6 /

In [ ]:
do_epoch(trainer, test_iter, is_train=False, name='Test:')

Test: Loss = 0.07067, Accuracy = 97.58%: 100%|██████████| 7/7 [00:00<00:00, 101.55it/s]


In [ ]:
from conlleval import evaluate

def eval_tagger(model, test_iter):
    true_seqs, pred_seqs = [], []

    model.eval()
    with torch.no_grad():
        for batch in test_iter:
            pred = model.forward(batch.tokens.transpose(0, 1)).transpose(1, 2).max(dim=1)[1].cpu().tolist()
            pred_seqs.extend([" ".join([tags_field.vocab.itos[elem] for elem in l if elem != 0]) for l in pred])
            true_seqs.extend([" ".join([tags_field.vocab.itos[elem] for elem in l if elem != 0]) for l in batch.tags.transpose(0, 1).cpu().tolist()])

    print('Precision = {:.2f}%, Recall = {:.2f}%, F1 = {:.2f}%'.format(*evaluate(true_seqs, pred_seqs, verbose=False)))

eval_tagger(model, test_iter)

Precision = 94.12%, Recall = 94.71%, F1 = 94.41%


## Multi-task learning

Реализуем модель, которая умеет сразу и предсказывать теги и интенты. Идея в том, что в этом всем есть общая информация, которая должна помочь как одной, так и другой задаче: зная интент, можно понять, какие слоты вообще могут быть, а зная слоты, можно угадать и интент.

#### **Задание 2.1**
Реализуйте объединенную модель.

In [ ]:
class SharedModel(nn.Module):
    def __init__(self, vocab_size, intents_count, tags_count, emb_dim=64,
                 lstm_hidden_dim=128, num_layers=1, dropout_p=0.2):
        super().__init__()
        # YOUR CODE HERE
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.dropout = nn.Dropout(0.2)
        self.lstm = nn.LSTM(emb_dim, lstm_hidden_dim, batch_first=True,
                            num_layers=num_layers, bidirectional=True)
        self.intent_linear = nn.Linear(2*lstm_hidden_dim, intents_count)
        self.tags_linear = nn.Linear(2*lstm_hidden_dim, tags_count)

    def forward(self, inputs):
        # YOUR CODE HERE
        emb = self.embedding(inputs)
        drp = self.dropout(emb)
        otp, (hidden_state, _) = self.lstm(drp)
        hid = torch.cat((hidden_state[0], hidden_state[1]), dim=1)
        tag_output = self.tags_linear(otp)
        intent_output = self.intent_linear(hid)
        return tag_output, intent_output

#### **Задание 2.2**
Допишите SharedModelTrainer

In [ ]:
class SharedModelTrainer():
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        self.epoch_loss = 0
        self.tag_correct_count, self.tag_total_count = 0, 0
        self.intent_correct_count, self.intent_total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        self.model.train(is_train)
        
    def on_epoch_end(self):
        return '{:>5s} Loss = {:.5f}, Tags accuracy = {:.2%}, Intents accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.tag_correct_count / self.tag_total_count, 
            self.intent_correct_count / self.intent_total_count
        )
        
    def on_batch(self, batch):
        # YOUR CODE HERE
        tag_logits, int_logits = self.model(batch.tokens.transpose(0, 1))
        #tag
        tag_true = batch.tags.transpose(0, 1)
        tag_loss = self.criterion(tag_logits.transpose(1, 2), tag_true)
        tag_predict = torch.max(tag_logits, axis=2).indices
        predicted_tags = torch.max(tag_logits, axis=2).indices
        self.tag_total_count += torch.sum(tag_true!=0).item()
        self.tag_correct_count += torch.sum(predicted_tags==tag_true).item() - torch.sum(tag_true==0).item()
        #int
        int_true = batch.intent
        int_loss = self.criterion(int_logits, int_true)
        predicted_intent = torch.max(int_logits, axis=1)[1]
        self.intent_total_count += predicted_intent.size(0)
        self.intent_correct_count += torch.sum(predicted_intent == batch.intent).item()
        #common
        loss = tag_loss + int_loss

        if self.is_train:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
        self.epoch_loss += loss.item()

In [ ]:
model = SharedModel(vocab_size=len(tokens_field.vocab), intents_count=len(intent_field.vocab),
                    tags_count=len(tags_field.vocab)).to(DEVICE)
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())
trainer = SharedModelTrainer(model, criterion, optimizer)
fit(trainer, train_iter, epochs_count=30, val_iter=val_iter)

[1 / 30] Train: Loss = 2.20143, Tags accuracy = 59.99%, Intents accuracy = 74.90%: 100%|██████████| 140/140 [00:01<00:00, 123.17it/s]
[1 / 30]   Val: Loss = 1.19105, Tags accuracy = 77.42%, Intents accuracy = 79.40%: 100%|██████████| 4/4 [00:00<00:00, 92.32it/s]
[2 / 30] Train: Loss = 0.94203, Tags accuracy = 82.74%, Intents accuracy = 86.69%: 100%|██████████| 140/140 [00:01<00:00, 106.33it/s]
[2 / 30]   Val: Loss = 0.75952, Tags accuracy = 87.85%, Intents accuracy = 87.00%: 100%|██████████| 4/4 [00:00<00:00, 82.08it/s]
[3 / 30] Train: Loss = 0.60574, Tags accuracy = 89.92%, Intents accuracy = 90.78%: 100%|██████████| 140/140 [00:01<00:00, 99.58it/s] 
[3 / 30]   Val: Loss = 0.57435, Tags accuracy = 91.81%, Intents accuracy = 89.80%: 100%|██████████| 4/4 [00:00<00:00, 62.69it/s]
[4 / 30] Train: Loss = 0.45551, Tags accuracy = 92.89%, Intents accuracy = 92.32%: 100%|██████████| 140/140 [00:01<00:00, 95.81it/s]
[4 / 30]   Val: Loss = 0.49700, Tags accuracy = 93.49%, Intents accuracy = 91.

In [ ]:
do_epoch(trainer, test_iter, is_train=False, name='Test:')

Test: Loss = 0.35534, Tags accuracy = 97.32%, Intents accuracy = 94.85%: 100%|██████████| 7/7 [00:00<00:00, 96.77it/s] 


In [ ]:
from conlleval import evaluate

def eval_tagger(model, test_iter):
    true_seqs, pred_seqs = [], []

    model.eval()
    with torch.no_grad():
        for batch in test_iter:
            pred = model.forward(batch.tokens.transpose(0, 1))[0].transpose(1, 2).max(dim=1)[1].cpu().tolist()
            true = batch.tags.transpose(0, 1).cpu().tolist()
            pred_seqs.extend([" ".join([tags_field.vocab.itos[elem] for elem in l if elem != 0]) for l in pred])
            true_seqs.extend([" ".join([tags_field.vocab.itos[elem] for elem in l if elem != 0]) for l in true])

    print('Precision = {:.2f}%, Recall = {:.2f}%, F1 = {:.2f}%'.format(*evaluate(true_seqs, pred_seqs, verbose=False)))

eval_tagger(model, test_iter)

Precision = 93.84%, Recall = 93.41%, F1 = 93.62%


 ## Асинхронное обучение

Идея описана в статье [A Bi-model based RNN Semantic Frame Parsing Model for Intent Detection and Slot Filling](http://aclweb.org/anthology/N18-2050).

<img src="https://i.ibb.co/qrgVSqF/2018-11-27-2-11-17.png" width="600"/>

Основное отличие от того, что уже реализовали в том, в каком порядке все оптимизируется. Вместо объединенного обучения всех слоев, сети для теггера и для классификатора обучаются отдельно.

На каждом шаге обучения генерируются последовательности скрытых состояний $h^1$ и $h^2$ - для классификатора и для теггера.

Дальше сначала считаются потери от предсказания интента и делается шаг оптимизатора, а затем потери от предсказания теггов - и опять шаг оптимизатора.

#### **Задание 3.1**
Реализуйте асинхронное обучение совместной модели

In [ ]:
class AsyncSharedModel(nn.Module):
    def __init__(self, vocab_size, intents_count, tags_count, emb_dim=65,
                 lstm_hidden_dim=128, num_layers=1, dropout_p=0.2):
        super().__init__()
        # YOUR CODE HERE
        # intents
        self.int_emb = nn.Embedding(vocab_size, emb_dim)
        self.drp = nn.Dropout(0.2)
        self.int_lstm = nn.LSTM(emb_dim, lstm_hidden_dim, batch_first=True, num_layers=num_layers, bidirectional=True)
        self.int_linear = nn.Linear(2*lstm_hidden_dim, intents_count)

        self.tag_emb = nn.Embedding(vocab_size, emb_dim)
        self.tag_lstm = nn.LSTM(emb_dim, lstm_hidden_dim, batch_first=True, num_layers=num_layers, bidirectional=True)
        self.tag_linear = nn.Linear(2*lstm_hidden_dim, tags_count)

    def forward(self, inputs):
        # YOUR CODE HERE
        #1) intents
        int_proj = self.int_emb(inputs)
        int_otp, (int_hid, _) = self.int_lstm(int_proj)
        int_hid = torch.cat((int_hid[0], int_hid[1]), dim=1)
        int_hid = self.drp(int_hid)
        intent_output = self.int_linear(int_hid)

        #2) tags
        tag_proj = self.int_emb(inputs)
        tag_otp, (tag_hid, _) = self.int_lstm(tag_proj)
        tag_hid = torch.cat((tag_hid[0], tag_hid[1]), dim=1)
        tag_otp = self.drp(tag_otp)
        tag_output = self.tag_linear(tag_otp)
        return tag_output, intent_output

In [ ]:
class AsyncSharedModelTrainer():
    def __init__(self, model, criterion, tags_optimizer, intent_optimizer):
        self.model = model
        self.criterion = criterion
        self.tags_optimizer = tags_optimizer
        self.intent_optimizer = intent_optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        self.epoch_loss = 0
        self.tag_correct_count, self.tag_total_count = 0, 0
        self.intent_correct_count, self.intent_total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        self.model.train(is_train)
        
    def on_epoch_end(self):
        return '{:>5s} Loss = {:.5f}, Tags accuracy = {:.2%}, Intents accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.tag_correct_count / self.tag_total_count, 
            self.intent_correct_count / self.intent_total_count
        )
        
    def on_batch(self, batch):
        # YOUR CODE HERE
        tokens = batch.tokens.transpose(0,1)
        tag_logits, int_logits = self.model(tokens)
        # intents
        int_loss = self.criterion(int_logits, batch.intent)
        int_pred = torch.max(int_logits, dim=1).indices
        self.intent_correct_count += torch.sum(int_pred==batch.intent).item()
        self.intent_total_count += batch.intent.size()[0]
        #tags
        tag_pred = torch.max(tag_logits, dim=2).indices
        tag_logits = tag_logits.transpose(1,2)
        tags = batch.tags.transpose(0,1)
        tag_loss = self.criterion(tag_logits, tags)
        self.tag_correct_count += torch.sum(tag_pred==tags).item() - torch.sum(tags==0).item()
        self.tag_total_count += torch.sum(tags!=0).item()
        #comm
        if self.is_train:
            self.intent_optimizer.zero_grad()
            self.tags_optimizer.zero_grad()
            int_loss.backward(retain_graph=True)
            tag_loss.backward(retain_graph=True)
            self.intent_optimizer.step()
            self.tags_optimizer.step()
        self.epoch_loss += int_loss.item() + tag_loss.item()

Затем их нужно передать в отдельные оптимизаторы и учить отдельно.

*Еще, может быть, пригодится retain_graph параметр метода backward()*.

In [ ]:
model = AsyncSharedModel(vocab_size=len(tokens_field.vocab), intents_count=len(intent_field.vocab),
                         tags_count=len(tags_field.vocab)).to(DEVICE)
criterion = nn.CrossEntropyLoss().to(DEVICE)
tags_parameters = [param for name, param in model.named_parameters() if not 'int_' in name]
intent_parameters = [param for name, param in model.named_parameters() if not 'tag_' in name]
tags_optimizer = optim.Adam(tags_parameters)
intent_optimizer = optim.Adam(intent_parameters)
trainer = AsyncSharedModelTrainer(model, criterion, tags_optimizer, intent_optimizer)
fit(trainer, train_iter, epochs_count=30, val_iter=val_iter)

[1 / 30] Train: Loss = 2.25521, Tags accuracy = 56.62%, Intents accuracy = 74.23%: 100%|██████████| 140/140 [00:01<00:00, 99.06it/s] 
[1 / 30]   Val: Loss = 1.27366, Tags accuracy = 77.31%, Intents accuracy = 79.20%: 100%|██████████| 4/4 [00:00<00:00, 84.40it/s]
[2 / 30] Train: Loss = 0.94051, Tags accuracy = 83.91%, Intents accuracy = 85.57%: 100%|██████████| 140/140 [00:01<00:00, 105.06it/s]
[2 / 30]   Val: Loss = 0.76781, Tags accuracy = 88.06%, Intents accuracy = 86.80%: 100%|██████████| 4/4 [00:00<00:00, 87.98it/s]
[3 / 30] Train: Loss = 0.54218, Tags accuracy = 91.24%, Intents accuracy = 91.76%: 100%|██████████| 140/140 [00:01<00:00, 105.12it/s]
[3 / 30]   Val: Loss = 0.46966, Tags accuracy = 91.76%, Intents accuracy = 90.20%: 100%|██████████| 4/4 [00:00<00:00, 78.59it/s]
[4 / 30] Train: Loss = 0.36085, Tags accuracy = 94.02%, Intents accuracy = 94.37%: 100%|██████████| 140/140 [00:01<00:00, 91.26it/s]
[4 / 30]   Val: Loss = 0.35633, Tags accuracy = 94.00%, Intents accuracy = 93.

In [ ]:
do_epoch(trainer, test_iter, is_train=False, name='Test:')

Test: Loss = 0.38197, Tags accuracy = 97.20%, Intents accuracy = 94.74%: 100%|██████████| 7/7 [00:00<00:00, 84.07it/s]


In [ ]:
from conlleval import evaluate

def eval_tagger(model, test_iter):
    true_seqs, pred_seqs = [], []

    model.eval()
    with torch.no_grad():
        for batch in test_iter:
            pred = model.forward(batch.tokens.transpose(0, 1))[0].transpose(1, 2).max(dim=1)[1].cpu().tolist()
            pred_seqs.extend([" ".join([tags_field.vocab.itos[elem] for elem in l if elem != 0]) for l in pred])
            true_seqs.extend([" ".join([tags_field.vocab.itos[elem] for elem in l if elem != 0]) for l in batch.tags.transpose(0, 1).cpu().tolist()])

    print('Precision = {:.2f}%, Recall = {:.2f}%, F1 = {:.2f}%'.format(*evaluate(true_seqs, pred_seqs, verbose=False)))

eval_tagger(model, test_iter)

Precision = 93.53%, Recall = 93.27%, F1 = 93.40%


#### **Задание 3.2**
Посмотрите на параметры в статье и попробуйте добиться похожего качества.

статья: https://www.csie.ntu.edu.tw/~yvchen/doc/NAACL18_SlotGated.pdf

Хорошая идея - использовать не просто независимые предсказания тегов, а декодер над ними:

архитектура:
![](https://i.ibb.co/qrgVSqF/2018-11-27-2-11-17.png =600x)

По сути тут добавляется просто еще слой RNN - на этот раз однонаправленной. При этом его вход в случае предсказания тегов - это предыдущий тег, предыдущее скрытое состояние и скрытые состояния из энкодеров теггов и интента. Для интента - простая RNN.

##### Интенты

In [ ]:
class IntentRNNModel(nn.Module):
    def __init__(self, vocab_size, intents_count, emb_dim=64,
                 lstm_hidden_dim=128, num_layers=1, dropout_p=0.2):
        super().__init__()
        # YOUR CODE HERE
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.enc_lstm = nn.LSTM(emb_dim, lstm_hidden_dim, batch_first=True,
                            num_layers=num_layers, bidirectional=True)
        self.dec_lstm = nn.LSTM(2*lstm_hidden_dim, lstm_hidden_dim, batch_first=True, num_layers=num_layers)
        self.linear = nn.Linear(lstm_hidden_dim, intents_count)

    def forward(self, inputs):
        # YOUR CODE HERE
        emb = self.embedding(inputs)
        drp = self.dropout(emb)
        enc_ountput, (enc_hidden_state, _) = self.enc_lstm(drp)
        hidden = torch.cat((enc_hidden_state[0], enc_hidden_state[1]), dim=1)
        dec_ountput, (dec_hidden_state, _) = self.dec_lstm(enc_ountput)
        output = self.linear(dec_hidden_state[0])
        return output

In [ ]:
model = IntentRNNModel(vocab_size=len(tokens_field.vocab), intents_count=len(intent_field.vocab)).to(DEVICE)
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())
trainer = ModelTrainer(model, criterion, optimizer)
fit(trainer, train_iter, epochs_count=30, val_iter=val_iter)

[1 / 30] Train: Loss = 1.24113, Accuracy = 72.71%: 100%|██████████| 140/140 [00:01<00:00, 112.09it/s]
[1 / 30]   Val: Loss = 1.25193, Accuracy = 71.40%: 100%|██████████| 4/4 [00:00<00:00, 94.01it/s] 
[2 / 30] Train: Loss = 1.13294, Accuracy = 73.89%: 100%|██████████| 140/140 [00:01<00:00, 119.55it/s]
[2 / 30]   Val: Loss = 1.24894, Accuracy = 71.40%: 100%|██████████| 4/4 [00:00<00:00, 83.27it/s]
[3 / 30] Train: Loss = 1.04045, Accuracy = 74.45%: 100%|██████████| 140/140 [00:01<00:00, 120.26it/s]
[3 / 30]   Val: Loss = 1.05034, Accuracy = 73.20%: 100%|██████████| 4/4 [00:00<00:00, 95.74it/s]
[4 / 30] Train: Loss = 0.85266, Accuracy = 75.97%: 100%|██████████| 140/140 [00:01<00:00, 125.57it/s]
[4 / 30]   Val: Loss = 0.93331, Accuracy = 75.40%: 100%|██████████| 4/4 [00:00<00:00, 100.63it/s]
[5 / 30] Train: Loss = 0.71674, Accuracy = 81.38%: 100%|██████████| 140/140 [00:01<00:00, 124.21it/s]
[5 / 30]   Val: Loss = 0.84196, Accuracy = 78.60%: 100%|██████████| 4/4 [00:00<00:00, 93.91it/s]
[6 

In [ ]:
do_epoch(trainer, test_iter, is_train=False, name='Test:')

Test: Loss = 0.32842, Accuracy = 95.18%: 100%|██████████| 7/7 [00:00<00:00, 103.60it/s]


##### Теггер

In [ ]:
class TaggerRNNModel(nn.Module):
    def __init__(self, vocab_size, tags_count, emb_dim=64,
                 lstm_hidden_dim=128, num_layers=1, dropout_p=0.2):
        super().__init__()
        # YOUR CODE HERE
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.dropout = nn.Dropout(0.2)
        self.enc_lstm = nn.LSTM(emb_dim, lstm_hidden_dim, batch_first=True,
                            num_layers=num_layers, bidirectional=True)
        self.dec_lstm = nn.LSTM(2*lstm_hidden_dim, lstm_hidden_dim, batch_first=True, num_layers=num_layers)
        self.linear = nn.Linear(lstm_hidden_dim, tags_count)

    def forward(self, inputs):
        emb = self.embedding(inputs)
        drp = self.dropout(emb)
        enc_ountput, (hidden_state, _) = self.enc_lstm(drp)
        dec_ountput, (hidden_state, _) = self.dec_lstm(enc_ountput)
        output = self.linear(dec_ountput)
        return output

In [ ]:
model = TaggerRNNModel(vocab_size=len(tokens_field.vocab), tags_count=len(tags_field.vocab)).to(DEVICE)
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())
trainer = TagModelTrainer(model, criterion, optimizer)
fit(trainer, train_iter, epochs_count=30, val_iter=val_iter)

[1 / 30] Train: Loss = 1.21541, Accuracy = 56.77%: 100%|██████████| 140/140 [00:01<00:00, 118.11it/s]
[1 / 30]   Val: Loss = 0.49616, Accuracy = 74.61%: 100%|██████████| 4/4 [00:00<00:00, 95.84it/s]
[2 / 30] Train: Loss = 0.47031, Accuracy = 80.55%: 100%|██████████| 140/140 [00:01<00:00, 102.48it/s]
[2 / 30]   Val: Loss = 0.28781, Accuracy = 84.01%: 100%|██████████| 4/4 [00:00<00:00, 83.18it/s]
[3 / 30] Train: Loss = 0.29098, Accuracy = 88.05%: 100%|██████████| 140/140 [00:01<00:00, 101.25it/s]
[3 / 30]   Val: Loss = 0.19691, Accuracy = 90.43%: 100%|██████████| 4/4 [00:00<00:00, 75.38it/s]
[4 / 30] Train: Loss = 0.19292, Accuracy = 92.20%: 100%|██████████| 140/140 [00:01<00:00, 91.37it/s]
[4 / 30]   Val: Loss = 0.13032, Accuracy = 93.09%: 100%|██████████| 4/4 [00:00<00:00, 78.16it/s]
[5 / 30] Train: Loss = 0.13993, Accuracy = 94.28%: 100%|██████████| 140/140 [00:01<00:00, 115.47it/s]
[5 / 30]   Val: Loss = 0.10328, Accuracy = 94.69%: 100%|██████████| 4/4 [00:00<00:00, 80.62it/s]
[6 / 3

In [ ]:
do_epoch(trainer, test_iter, is_train=False, name='Test:')

Test: Loss = 0.06675, Accuracy = 97.34%: 100%|██████████| 7/7 [00:00<00:00, 100.88it/s]


#### **Задание 4**
Посмотрите результаты на SNIPS

In [12]:
model = IntentClassifierModel(   vocab_size=len(snips_tokens_field.vocab), 
                              intents_count=len(snips_intent_field.vocab)).to(DEVICE)
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())
trainer = ModelTrainer(model, criterion, optimizer)
fit(trainer, snips_train_iter, epochs_count=30, val_iter=snips_val_iter)

[1 / 30] Train: Loss = 0.37172, Accuracy = 89.81%: 100%|██████████| 409/409 [00:02<00:00, 177.50it/s]
[1 / 30]   Val: Loss = 0.13436, Accuracy = 96.57%: 100%|██████████| 6/6 [00:00<00:00, 213.86it/s]
[2 / 30] Train: Loss = 0.08968, Accuracy = 97.25%: 100%|██████████| 409/409 [00:02<00:00, 185.55it/s]
[2 / 30]   Val: Loss = 0.10900, Accuracy = 96.71%: 100%|██████████| 6/6 [00:00<00:00, 212.58it/s]
[3 / 30] Train: Loss = 0.05022, Accuracy = 98.42%: 100%|██████████| 409/409 [00:02<00:00, 182.64it/s]
[3 / 30]   Val: Loss = 0.08238, Accuracy = 97.29%: 100%|██████████| 6/6 [00:00<00:00, 208.94it/s]
[4 / 30] Train: Loss = 0.03203, Accuracy = 99.08%: 100%|██████████| 409/409 [00:02<00:00, 186.51it/s]
[4 / 30]   Val: Loss = 0.11307, Accuracy = 96.43%: 100%|██████████| 6/6 [00:00<00:00, 221.44it/s]
[5 / 30] Train: Loss = 0.02239, Accuracy = 99.30%: 100%|██████████| 409/409 [00:02<00:00, 185.71it/s]
[5 / 30]   Val: Loss = 0.07490, Accuracy = 97.14%: 100%|██████████| 6/6 [00:00<00:00, 211.77it/s]


In [13]:
do_epoch(trainer, snips_test_iter, is_train=False, name='Test:')

Test: Loss = 0.18891, Accuracy = 96.43%: 100%|██████████| 6/6 [00:00<00:00, 202.75it/s]


## Async Multi-task Learning for POS Tagging

Ещё одна статья: [Morphosyntactic Tagging with a Meta-BiLSTM Model over Context Sensitive Token Encodings](https://arxiv.org/pdf/1805.08237.pdf)

Архитектура там такая:

<img src="https://i.ibb.co/0nSX6CC/2018-11-27-9-26-15.png" width="400"/>

Multi-task задача - обучение отдельных классификаторов более низкого уровня (над символами и словами) для предсказания тегов отдельными оптимизаторами.

## DeepPavlov go_bot

http://docs.deeppavlov.ai/en/master/features/skills/go_bot.html

In [ ]:
# !pip install deeppavlov==0.14.0
!python -m deeppavlov install gobot_dstc2

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
from deeppavlov import build_model, configs

bot1 = build_model(configs.go_bot.gobot_dstc2, download=True)

bot1(['hi, i want restaurant in the cheap pricerange'])
bot1(['bye'])

Поддробные туториалы:

Simple: https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_tutorial.ipynb

Extended: https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_extended_tutorial.ipynb

# Дополнительные материалы

## Статьи
A Bi-model based RNN Semantic Frame Parsing Model for Intent Detection and Slot Filling, 2018 [[pdf]](http://aclweb.org/anthology/N18-2050)

Slot-Gated Modeling for Joint Slot Filling and Intent Prediction, 2018 [[pdf]](http://aclweb.org/anthology/N18-2118) 

Morphosyntactic Tagging with a Meta-BiLSTM Model over Context Sensitive Token Encodings, 2018 [[pdf]](https://arxiv.org/pdf/1805.08237.pdf)

BERT for Joint Intent Classification and Slot Filling
 [[pdf]](https://arxiv.org/pdf/1902.10909.pdf)

## Блоги
[Как устроена Алиса](https://habr.com/company/yandex/blog/349372/)  